In [47]:
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import metrics, Input
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [48]:
METRICS = [metrics.RootMeanSquaredError(name='rms'), metrics.MeanAbsoluteError(name='mae')]

In [74]:
ENCODING_DIM = 16 #Desired Dimension
BATCH_SIZE = 64
EPOCHS = 50

In [75]:
def make_and_train_autoencoder(X_train, metrics=METRICS):
    
    len_input_output = X_train.shape[-1]
    input_ = Input(shape=(len_input_output,))
    encoded = Dense(units=ENCODING_DIM*2, activation="relu")(input_)
    bottleneck = Dense(units=ENCODING_DIM, 
                       activation="relu")(encoded)
    decoded = Dense(units=ENCODING_DIM*2, 
                    activation="relu")(bottleneck)
    output = Dense(units=len_input_output, 
                    activation="linear")(decoded)
    #Training is performed on the entire autoencoder
    autoencoder = Model(inputs=input_, outputs=output)
    autoencoder.compile(optimizer='adam', loss='mean_squared_error',
                        metrics=[metrics])
    autoencoder.fit(X_train, X_train,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS)
    #Use only the encoder part for dimensionality reduction
    encoder = Model(inputs=input_, outputs=bottleneck)
    return autoencoder, encoder

In [76]:
df = pd.read_csv('training_ROS.csv')
qw = pd.read_csv('validation_set.csv')

In [77]:
X, y = df.iloc[:,:-1], df.iloc[:, -1]

In [78]:
auto, encode = make_and_train_autoencoder(X, metrics=METRICS)

Epoch 1/50
512/512 [==============================] - 7s 9ms/step - loss: 0.0074 - rms: 0.0726 - mae: 0.0382
Epoch 2/50
512/512 [==============================] - 4s 7ms/step - loss: 9.1966e-04 - rms: 0.0303 - mae: 0.0174
Epoch 3/50
512/512 [==============================] - 4s 7ms/step - loss: 5.5984e-04 - rms: 0.0237 - mae: 0.0124
Epoch 4/50
512/512 [==============================] - 4s 8ms/step - loss: 3.6246e-04 - rms: 0.0190 - mae: 0.0094
Epoch 5/50
512/512 [==============================] - 3s 7ms/step - loss: 2.6789e-04 - rms: 0.0164 - mae: 0.0076
Epoch 6/50
512/512 [==============================] - 3s 7ms/step - loss: 2.3918e-04 - rms: 0.0155 - mae: 0.0070
Epoch 7/50
512/512 [==============================] - 3s 7ms/step - loss: 2.2582e-04 - rms: 0.0150 - mae: 0.0063
Epoch 8/50
512/512 [==============================] - 3s 7ms/step - loss: 2.1835e-04 - rms: 0.0148 - mae: 0.0059
Epoch 9/50
512/512 [==============================] - 3s 7ms/step - loss: 2.1488e-04 - rms: 0.0147 -

In [85]:
q = encode.predict(X)

In [86]:
q.shape

(32710, 16)

In [87]:
q = np.array(q)
df = pd.DataFrame(data=q,index=None,columns=None)

In [82]:
df['label'] = y
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,label
0,0.283456,0.577016,0.423019,0.0,0.250961,0.202846,0.481518,0.244313,0.0,0.454076,0.393439,0.548409,0.0,0.185507,0.648040,0.0,0.0
1,0.493598,0.541534,0.530520,0.0,0.181428,0.399625,0.830677,0.337497,0.0,0.638950,0.381134,0.729409,0.0,0.136083,0.676449,0.0,1.0
2,0.403154,0.321343,0.285773,0.0,0.303656,0.857309,0.391485,0.756702,0.0,0.486127,0.419870,0.800637,0.0,0.240612,0.818876,0.0,0.0
3,0.500504,0.489318,0.190704,0.0,0.183143,0.559673,0.443194,0.312199,0.0,0.369171,0.542243,0.441686,0.0,0.236049,0.453401,0.0,0.0
4,0.486707,0.445970,0.184550,0.0,0.146938,0.422056,0.190994,0.265513,0.0,0.326501,0.416364,0.315365,0.0,0.188302,0.372284,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32705,0.535559,0.441531,0.155430,0.0,0.098877,0.410986,0.186439,0.152110,0.0,0.420630,0.293837,0.362307,0.0,0.125869,0.459955,0.0,1.0
32706,0.406270,0.635256,0.557693,0.0,0.418377,0.540109,0.657915,0.913499,0.0,0.660840,0.443102,0.987366,0.0,0.283028,1.193918,0.0,1.0
32707,0.308616,0.412434,0.616408,0.0,0.407357,0.596448,0.612854,0.690864,0.0,0.541341,0.506517,1.226618,0.0,0.197294,1.111643,0.0,1.0
32708,0.309377,0.101572,0.228378,0.0,0.229550,0.640677,0.003570,0.597411,0.0,0.260949,0.359638,0.665072,0.0,0.100872,0.632721,0.0,1.0


In [84]:
df.to_csv('training_ros_autoencoded_16.csv', index=False)

In [88]:
#### CNN autoencode
df = pd.read_csv('training_CNN.csv')
X, y = df.iloc[:,:-1], df.iloc[:, -1]
auto, encode = make_and_train_autoencoder(X, metrics=METRICS)
q = encode.predict(X)
w = np.array(q)
df = pd.DataFrame(data=w,index=None,columns=None)
df['label'] = y
df.to_csv('training_cnn_autoencoded_16.csv', index=False)

Epoch 1/50
156/156 [==============================] - 3s 7ms/step - loss: 0.0246 - rms: 0.0768 - mae: 0.0262
Epoch 2/50
156/156 [==============================] - 1s 7ms/step - loss: 0.0047 - rms: 0.0684 - mae: 0.0447
Epoch 3/50
156/156 [==============================] - 1s 7ms/step - loss: 0.0019 - rms: 0.0435 - mae: 0.0278
Epoch 4/50
156/156 [==============================] - 1s 7ms/step - loss: 0.0012 - rms: 0.0340 - mae: 0.0202
Epoch 5/50
156/156 [==============================] - 1s 9ms/step - loss: 9.1826e-04 - rms: 0.0303 - mae: 0.0171
Epoch 6/50
156/156 [==============================] - 1s 7ms/step - loss: 7.5311e-04 - rms: 0.0274 - mae: 0.0150
Epoch 7/50
156/156 [==============================] - 1s 9ms/step - loss: 5.9889e-04 - rms: 0.0245 - mae: 0.0131
Epoch 8/50
156/156 [==============================] - 1s 7ms/step - loss: 4.8097e-04 - rms: 0.0219 - mae: 0.0112
Epoch 9/50
156/156 [==============================] - 1s 7ms/step - loss: 4.1645e-04 - rms: 0.0204 - mae: 0.0102

In [89]:
#### CNN autoencode
df = pd.read_csv('training_RUS.csv')
X, y = df.iloc[:,:-1], df.iloc[:, -1]
auto, encode = make_and_train_autoencoder(X, metrics=METRICS)
q = encode.predict(X)
w = np.array(q)
df = pd.DataFrame(data=w,index=None,columns=None)
df['label'] = y
df.to_csv('training_rus_autoencoded_16.csv', index=False)

Epoch 1/50
146/146 [==============================] - 3s 7ms/step - loss: 0.0237 - rms: 0.1071 - mae: 0.0477
Epoch 2/50
146/146 [==============================] - 1s 7ms/step - loss: 0.0046 - rms: 0.0680 - mae: 0.0456
Epoch 3/50
146/146 [==============================] - 1s 7ms/step - loss: 0.0021 - rms: 0.0463 - mae: 0.0299
Epoch 4/50
146/146 [==============================] - 1s 7ms/step - loss: 0.0014 - rms: 0.0369 - mae: 0.0230
Epoch 5/50
146/146 [==============================] - 1s 7ms/step - loss: 9.7783e-04 - rms: 0.0313 - mae: 0.0186
Epoch 6/50
146/146 [==============================] - 1s 7ms/step - loss: 7.4698e-04 - rms: 0.0273 - mae: 0.0156
Epoch 7/50
146/146 [==============================] - 1s 7ms/step - loss: 6.1093e-04 - rms: 0.0247 - mae: 0.0136
Epoch 8/50
146/146 [==============================] - 1s 7ms/step - loss: 5.3034e-04 - rms: 0.0230 - mae: 0.0122
Epoch 9/50
146/146 [==============================] - 1s 7ms/step - loss: 4.7710e-04 - rms: 0.0218 - mae: 0.0111

In [90]:
#### CNN autoencode
df = pd.read_csv('training_smote.csv')
X, y = df.iloc[:,:-1], df.iloc[:, -1]
auto, encode = make_and_train_autoencoder(X, metrics=METRICS)
q = encode.predict(X)
w = np.array(q)
df = pd.DataFrame(data=w,index=None,columns=None)
df['label'] = y
df.to_csv('training_smote_autoencoded_16.csv', index=False)

Epoch 1/50
512/512 [==============================] - 7s 9ms/step - loss: 0.0075 - rms: 0.0768 - mae: 0.0390
Epoch 2/50
512/512 [==============================] - 4s 7ms/step - loss: 6.4634e-04 - rms: 0.0254 - mae: 0.0147
Epoch 3/50
512/512 [==============================] - 4s 7ms/step - loss: 2.9511e-04 - rms: 0.0172 - mae: 0.0089
Epoch 4/50
512/512 [==============================] - 4s 7ms/step - loss: 2.1516e-04 - rms: 0.0147 - mae: 0.0072
Epoch 5/50
512/512 [==============================] - 4s 7ms/step - loss: 1.6579e-04 - rms: 0.0129 - mae: 0.0061
Epoch 6/50
512/512 [==============================] - 4s 7ms/step - loss: 1.3538e-04 - rms: 0.0116 - mae: 0.0051
Epoch 7/50
512/512 [==============================] - 4s 7ms/step - loss: 1.1969e-04 - rms: 0.0109 - mae: 0.0047
Epoch 8/50
512/512 [==============================] - 4s 7ms/step - loss: 1.1264e-04 - rms: 0.0106 - mae: 0.0045
Epoch 9/50
512/512 [==============================] - 4s 7ms/step - loss: 1.0944e-04 - rms: 0.0105 -